# TESTE PRÁTICO DE LINGUAGEM DE CONSULTA ESTRUTURADA (SQL)
Para a realização das atividades, você deverá executar as instruções abaixo para importar as bibliotecas e funções necessárias para criar a base de dados.

Ao final, esperamos que você desenvolva os códigos selecionados e entregue o notebook com as consultas SQL desenvolvidas e o arquivo .db criado.

O teste importará três arquivos cruciais para o desenvolvimento dessa atividade. Todos estão no Github (não mude as URL's).

*   usuarios.csv
*   produtos.csv
*   vendas.csv

## O teste foi desenvolvido para ser executado com o SQLite3 e no Google Colab (Ubuntu 18.04). Você deverá utilizar consultas no padrão ANSI. Os comandos necessários podem ser encontrados em:

https://www.sqlite.org/docs.html

https://www.tutorialspoint.com/sqlite/index.htm



# AVISO!



1.   Execute o código abaixo para atualizar a biblioteca do SQLite3 utilizada para essa atividade
2.   **Ignore as mensagens de erro sobre perda de conectividade na instrução abaixo (o kernel será reiniciado para atualização do SQLite3)**


In [ ]:
import os

# BAIXAR A ÚLTIMA RELEASE DO COLAB, DESCOMPACTAR E INSTALAR NO DIRETÓRIO DO 
# PYTHON 3.7 (VERSÃO USADA PELO COLAB NESSE NOTEBOOK)
!curl https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release | tar xz
%cd sqlite/
!./configure
!make sqlite3.c
%cd /content/sqlite/
!npx degit coleifer/pysqlite3 -f
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build
!cp build/lib.linux-x86_64-3.7/pysqlite3/_sqlite3.cpython-37m-x86_64-linux-gnu.so \
     /usr/lib/python3.7/lib-dynload/

# REINICIANDO O KERNEL DO COLAB PARA IMPORTAR A BIBLIOTECA ATUALIZADA
os.kill(os.getpid(), 9)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.1M  100 11.1M    0     0  5076k      0  0:00:02  0:00:02 --:--:-- 5076k
/content/sqlite
checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking for gcc... gcc
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether we are using the GNU C compiler... yes
checking whether gcc accepts -g... yes
checking for gcc option to accept ISO C89... none needed
checking for a sed that does not truncate output... /bin/sed
checking for grep that handles long lines and -e... /bin/grep
checking for egrep... /bin/grep -E
checking for fgrep... /bin/grep -F
checking for ld used by gcc... /usr/bin/ld
checkin

# INSTRUÇÕES PARA REALIZAR AS CONSULTAS
### PARA CONSULTAR DADOS, SIGA OS PASSOS ABAIXO:

#### Abrir conexão:
    conexao_sql = sqlite3.connect(banco)
#### Abrir cursor para executar consulta no banco:
    cursor = conexao_sql.cursor()
#### Criar uma consulta:
    consulta = '''
        Coloque aqui a sua instrução SQL;
    '''
    cursor.execute(consulta)
#### Para visualizar o resultado da sua consulta:
    for linha in cursor:
      print(linha)

      pd.read_sql(consulta, conexao_sql) # minha maneira de visualizar a consulta
#### Para fechar a conexão:
    conexao_sql.close()

### LEMBRE-SE DE QUE PYTHON É SENSÍVEL A IDENTAÇÃO DE CÓDIGO!

# IMPORTANDO BIBLIOTECAS NECESSÁRIAS PARA EXECUTAR OS CÓDIGOS EM SQL

In [1]:
import sqlite3
import csv
import requests
from pathlib import Path
from urllib.parse import urlparse

import pandas as pd # para melhor visualizar o resultado das queries

# DEFINIÇÃO DE VARIÁVEIS FIXAS PARA O PROJETO

In [2]:
banco = "teste_sql.db"

# FUNÇÃO PARA INSERIR REGISTROS NO BANCO DE DADOS

In [3]:
# Recebe como parâmetro
# 1. Nome do banco de dados
# 2. Nome da tabela que receberá os dados
# 3. Lista das colunas da tabela
# 4. Dados que serão inseridos

def inserir_registros(banco, tabela, lista_colunas, arquivo):
  conexao_sql = sqlite3.connect(banco)
  cursor = conexao_sql.cursor()
  dados = open(arquivo)
  conteudo_dados = csv.reader(dados)
  num_colunas = 1 if lista_colunas.count(',') == 1 else lista_colunas.count(',')
  string_insert = 'INSERT INTO ' + tabela + ' (' + lista_colunas + ') VALUES (?' + ('' if num_colunas == 1 else ', ?' * (num_colunas)) + ')'
  cursor.executemany(string_insert, conteudo_dados)
  conexao_sql.commit()
  conexao_sql.close()

# FUNÇÃO PARA BAIXAR A BASE DE DADOS UTILIZADA PARA DESENVOLVIMENTO DAS ATIVIDADES

In [4]:
# Recebe como parâmetro:
# 1. Nome do arquivo com a extensão (Ex: base.csv)
# 2. URL do site onde o dado está localizado (Ex: 'https://drive.google.com/user_teste/folder_dataset/file.csv'):

def baixar_dados(nome_arquivo, url):
  requisicao = requests.get(url)
  conteudo = requisicao.content
  arquivo_csv = open(nome_arquivo, 'wb')
  arquivo_csv.write(conteudo)
  arquivo_csv.close()

# CRIAÇÃO DE CONEXÃO COM O BANCO SQLITE3 PARA DESENVOLVIMENTO DA ATIVIDADE

In [5]:
# Abre conexão com o banco. Se o arquivo não existir, cria a base.
conexao_sql = sqlite3.connect(banco)
conexao_sql.close()

# CRIAÇÃO DAS TABELAS PARA REALIZAÇÃO DAS ATIVIDADES

### TABELA DE USUÁRIOS

In [6]:
url = 'https://raw.githubusercontent.com/A3Data/testes_praticos/main/sql/base_dados/usuarios.csv'
nome_arquivo = Path(urlparse(url).path).name
baixar_dados(nome_arquivo, url)

conexao_sql = sqlite3.connect(banco)
# CRIAR A TABELA
conexao_sql.execute('''
CREATE TABLE IF NOT EXISTS usuarios
(
  cod_usuario INTEGER NOT NULL,
  data_cadastro DATE NOT NULL,
  faixa_etaria TEXT,
  cidade TEXT NOT NULL,
  estado TEXT NOT NULL);'''
)
conexao_sql.commit()

# INSERIR OS REGISTROS NO BANCO DE DADOS
lista_campos = 'cod_usuario, data_cadastro, faixa_etaria, cidade, estado'
inserir_registros(banco, 'usuarios', lista_campos, nome_arquivo)

### TABELA DE PRODUTOS

In [7]:
url = 'https://raw.githubusercontent.com/A3Data/testes_praticos/main/sql/base_dados/produtos.csv'
nome_arquivo = Path(urlparse(url).path).name
baixar_dados(nome_arquivo, url)

conexao_sql = sqlite3.connect(banco)
arquivo_produtos = 'produtos.csv'
# CRIAR A TABELA
conexao_sql.execute('''
CREATE TABLE IF NOT EXISTS produtos
(
  cod_produto INTEGER NOT NULL,
  nome_produto TEXT NOT NULL,
  categoria_produto TEXT,
  valor_produto FLOAT);'''
)
conexao_sql.commit()

# INSERIR OS REGISTROS NO BANCO DE DADOS
lista_campos = 'cod_produto, nome_produto, categoria_produto, valor_produto'
inserir_registros(banco, 'produtos', lista_campos, arquivo_produtos)

### TABELA DE VENDAS

In [8]:
url = 'https://raw.githubusercontent.com/A3Data/testes_praticos/main/sql/base_dados/vendas.csv'
nome_arquivo = Path(urlparse(url).path).name
baixar_dados(nome_arquivo, url)

conexao_sql = sqlite3.connect(banco)
arquivo_vendas = 'vendas.csv'
# CRIAR A TABELA
conexao_sql.execute('''
CREATE TABLE IF NOT EXISTS vendas
(
  cod_usuario INTEGER NOT NULL,
  cod_produto INTEGER NOT NULL,
  data_compra DATE NOT NULL,
  quantidade INT NOT NULL,
  valor FLOAT NOT NULL);'''
)
conexao_sql.commit()

# INSERIR OS REGISTROS NO BANCO DE DADOS
lista_campos = 'cod_usuario, cod_produto, data_compra, quantidade, valor'
inserir_registros(banco, 'vendas', lista_campos, arquivo_vendas)

# Explorando as bases

---



In [9]:
conexao_sql = sqlite3.connect(banco)
cursor = conexao_sql.cursor()

In [10]:
consulta = '''SELECT COUNT(*) FROM usuarios;'''   

cursor.execute(consulta)
pd.read_sql(consulta, conexao_sql)

,COUNT(*)
0,5571


In [114]:
consulta = '''SELECT COUNT(*) FROM vendas;'''   

cursor.execute(consulta)
pd.read_sql(consulta, conexao_sql)

,COUNT(*)
0,20000


In [118]:
consulta = '''SELECT COUNT(*) FROM produtos;'''   

cursor.execute(consulta)
pd.read_sql(consulta, conexao_sql)

,COUNT(*)
0,21


# PERGUNTA 1:

### Escreva um comando em SQL que retorne a quantidade de vendas dos estados de Minas Gerais e São Paulo por ano e mês

Resposta: É necessário realizar uma query com Left Join para responder à pergunta, pois os registros de Estado não estão na mesma tabela dos registros de vendas. Com o Left Join, é possível estabelecer uma relação entre a venda realizada e o Estado referente a esse registro de venda.

A quantidade de vendas pode ser interpretada como o número de dias em que determinado usuário efetuou uma compra. Então, criei uma coluna que conta, agrupando por cod_usuario, as vezes em que o usuário efetuou uma compra ("vendas_por_usuario"). 

Por fim, coloquei o código acima em forma de subquery. Somei a coluna "quantidade_de_vendas", agrupando pela coluna "ano_mes_compra" e obtive o total de vendas efetuadas em Minas Gerais e São Paulo por ano e mês.

In [11]:
consulta = '''SELECT SUM(vendas_por_usuario) AS qtde_de_vendas, ano_mes_compra FROM (
                SELECT COUNT(v.data_compra) AS vendas_por_usuario, strftime('%Y-%m',data_compra) AS "ano_mes_compra", u.estado
                FROM vendas AS v
                LEFT JOIN usuarios AS u
                ON v.cod_usuario=u.cod_usuario
                WHERE estado='Minas Gerais' OR estado='São Paulo'
                GROUP BY v.cod_usuario)
              GROUP BY ano_mes_compra
              ;'''
        
cursor.execute(consulta)

pd.read_sql(consulta, conexao_sql)

,qtde_de_vendas,ano_mes_compra
0,5,2018-07
1,6,2018-10
2,9,2018-11
3,5,2018-12
4,5,2019-09
5,7,2019-11
6,3,2020-09
7,9,2021-02
8,6,2021-03
9,10,2021-04




---

CONFERINDO

---



In [123]:
consulta = '''SELECT COUNT(*) FROM (
                SELECT *
                FROM vendas AS v
                LEFT JOIN usuarios AS u
                ON v.cod_usuario=u.cod_usuario)
                WHERE estado='Minas Gerais' OR estado='São Paulo';
'''
        
cursor.execute(consulta)

pd.read_sql(consulta, conexao_sql)

,COUNT(*)
0,108


In [125]:
consulta = '''SELECT SUM(qtde_de_vendas) FROM (
                SELECT SUM(vendas_por_usuario) AS qtde_de_vendas, ano_mes_compra FROM (
                  SELECT COUNT(v.data_compra) AS vendas_por_usuario, strftime('%Y-%m',data_compra) AS "ano_mes_compra", u.estado
                  FROM vendas AS v
                  LEFT JOIN usuarios AS u
                  ON v.cod_usuario=u.cod_usuario
                  WHERE estado='Minas Gerais' OR estado='São Paulo'
                  GROUP BY v.cod_usuario)
                GROUP BY ano_mes_compra)
              ;'''
        
cursor.execute(consulta)

pd.read_sql(consulta, conexao_sql)

,SUM(qtde_de_vendas)
0,108


# PERGUNTA 2:

### Escreva um comando em SQL que retorne o ticket médio (valor médio de compra) e quantidade de compras realizadas dos usuários que fizeram compras entre 2019 e 2020 e que tiveram mais de 3 compras no período (lembre-se de contar todas as compras dos usuários que estão nessa condição!)

Resposta: A partir da tabela vendas, fez-se um filtro das vendas ocorridas nos anos de 2019 e 2020.



Em sequida, criei uma coluna que conta, agrupando por "cod_usuario", as vezes em que o usuário efetuou uma compra ("compras_por_usuario"). Com esta nova coluna, fez-se o filtro dos registros de compras efetuadas entre 2019 e 2020, com a "compras_por_usuario" maior que 3.

Por fim, criaram-se as 2 colunas que respondem à pergunta do problema. A coluna "qtde_de_compras", que é a soma da coluna "compras_por_usuario" e a coluna "ticket_medio", que é a divisão da soma do valor monetário das compras pela soma de todas as compras realizadas.

In [12]:
consulta = '''SELECT SUM(compras_por_usuario) AS qtde_de_compras, SUM(valor)/SUM(compras_por_usuario) AS ticket_medio FROM (
                SELECT * FROM (
                  SELECT cod_usuario, valor, strftime('%Y',data_compra) AS "ano_compra", COUNT(data_compra) AS "compras_por_usuario"
                  FROM vendas
                  WHERE (ano_compra BETWEEN '2019' AND '2020')
                  GROUP BY(cod_usuario))
                WHERE compras_por_usuario > 3);
'''
cursor.execute(consulta)

pd.read_sql(consulta, conexao_sql)

,qtde_de_compras,ticket_medio
0,6834,2190.103946


# PERGUNTA 3:

### Escreva um comando em SQL que retorne quais são os usuários por estado que mais compraram em todo o período analisado e, o número de compras realizadas, a quantidade total de itens comprados e valor total pago

Resposta: É necessário realizar uma query com Left Join para responder à pergunta, pois os registros de Estado não estão na mesma tabela dos registros de vendas.

A quantidade de vendas pode ser interpretada como o número de dias em que determinado usuário efetuou uma compra. Então, criei uma coluna que conta, agrupando por cod_usuario, os dias em que o usuário efetuou uma compra ("compras_por_usuario"). Em seguida, criei uma coluna que soma a quantidade de itens comprados, agrupando, também, pelo usuário ("qtde_total_itens_comprados"). Essa consulta foi disposta em uma subquery para que eu pudesse efetuar a próxima etapa.

A coluna compras_por_usuario recebeu a função MAX para que eu pudesse obter o usuário que fez o maior número de compras por estado. Por fim, exibi as colunas já calculadas para responder ao exercício.

In [13]:
consulta = ''' SELECT cod_usuario, estado, MAX(compras_por_usuario) AS num_compras_realizadas, qtde_total_itens_comprados, valor_total_pago FROM ( 
                SELECT v.cod_usuario, estado, COUNT(v.data_compra) AS compras_por_usuario, SUM(v.quantidade) AS qtde_total_itens_comprados, SUM(valor) AS valor_total_pago
                FROM vendas AS v
                LEFT JOIN usuarios as u
                ON u.cod_usuario=v.cod_usuario
                GROUP BY v.cod_usuario)
              GROUP BY estado
              ORDER BY valor_total_pago DESC;
'''
cursor.execute(consulta)

pd.read_sql(consulta, conexao_sql)

,cod_usuario,estado,num_compras_realizadas,qtde_total_itens_comprados,valor_total_pago
0,2811,Maranhão,13,117,184101.89
1,2340,Mato Grosso,9,112,165197.30
2,1074,Santa Catarina,9,66,159165.18
3,1170,None,15,192,156669.66
4,679,Mato Grosso do Sul,6,102,152271.96
5,700,Goiás,9,74,142389.09
6,1974,Piauí,5,77,139997.47
7,2843,Paraná,10,114,137228.01
8,3403,Rio Grande do Norte,10,94,136347.01
9,2646,Pernambuco,7,84,135556.70


# PERGUNTA 4:

### Escreva um comando em SQL que retorne quais são os 3 produtos mais comprados dos estados da região Sul e Sudeste, a quantidade de itens comprados, o valor total pago e a média de preço paga

Resposta: É necessário realizar uma query com Left Join para responder à pergunta, pois os registros de Estado não estão na mesma tabela dos registros de vendas (chave: cod_usuario). Além disso, também é necessário fazer um Inner Join para fazer a correspondência do código do produto com sua descrição (chave: cod_produto).

A seguir, somei a quantidade de itens comprados, agrupados por produto ("qtde_itens_comprados") e somei a coluna valor para obter o valor total da venda desses itens por produto ("valor_total_pago").

Em seguida, ordenei a consulta pela coluna "qtde_itens_comprados", de forma descendente e limitei a consulta para 3 resultados, a fim de visualizar os três produtos mais comprados em termos de quantidade de itens.

Por fim, adcionei a coluna final, "med_peco_paga", fazendo a divisão da coluna "valor_total_pago" pela coluna "qtde_itens_comprados". Obs.: valor coincide com o preço do produto (coluna "valor_produto" da tabela "produtos").

In [204]:
consulta = '''SELECT nome_produto, SUM(quantidade) AS qtde_itens_comprados, SUM(valor) AS valor_total_pago, (SUM(valor)/SUM(quantidade)) AS med_preco_paga FROM (
                SELECT * FROM (
                  SELECT * FROM vendas AS v
                  LEFT JOIN usuarios as u
                  ON v.cod_usuario=u.cod_usuario
                  WHERE estado IN ('Minas Gerais', 'São Paulo', 'Espírito Santo','Rio de Janeiro', 'Paraná', 'Santa Catarina', 'Rio Grande do Sul')
                  ) AS t1
                INNER JOIN produtos as p
                ON p.cod_produto=t1.cod_produto)
              GROUP BY cod_produto
              ORDER BY qtde_itens_comprados DESC
              LIMIT 3;
'''
cursor.execute(consulta)

pd.read_sql(consulta, conexao_sql)

,nome_produto,qtde_itens_comprados,valor_total_pago,med_preco_paga
0,Alcool em Gel 70% Johnson,280,1397.2,4.99
1,Escrivaninha em L,141,61617.0,437.00
2,Notebook Asus Intel Core i7,134,615180.6,4590.90


# PERGUNTA 5:

### Escreva um comando em SQL que retorne o ticket médio e o número total de usuários que fizeram pelo menos uma compra por faixa etária

Resposta: É necessário realizar uma query com Inner Join para responder à pergunta, pois os registros de faixa etária não não estão na mesma tabela dos registros de vendas (chave: cod_usuario).

A quantidade de compras pode ser interpretada como o número de dias em que determinado usuário efetuou uma compra. Então, criei uma coluna que conta, agrupando por cod_usuario, as vezes em que o usuário efetuou uma compra ("compras_por_usuario"). 

In [14]:
consulta = '''SELECT faixa_etaria, COUNT(cod_usuario) AS num_usuarios, SUM(valor_por_usuario)/SUM(compras_por_usuario) AS ticket_medio FROM (
                SELECT * FROM (
                  SELECT u.cod_usuario, u.faixa_etaria, COUNT(v.data_compra) AS compras_por_usuario, SUM(v.quantidade) AS itens_por_usuario, SUM(v.valor) AS valor_por_usuario 
                  FROM vendas AS v
                  LEFT JOIN usuarios AS u
                  ON u.cod_usuario=v.cod_usuario
                  GROUP BY u.cod_usuario)
                WHERE compras_por_usuario > 1)
              GROUP BY faixa_etaria
;
'''
cursor.execute(consulta)

pd.read_sql(consulta, conexao_sql)

,faixa_etaria,num_usuarios,ticket_medio
0,None,0,10782.242326
1,Entre 10 a 15 anos,7,8890.125349
2,Entre 16 a 21 anos,11,12725.947000
3,Entre 22 a 27 anos,10,12203.022969
4,Entre 28 a 36 anos,9,8233.037143
5,Entre 37 a 49 anos,6,16727.190714
6,Entre 50 a 61 anos,12,10849.475750
7,Entre 62 a 70 anos,7,10012.626410
8,Mais de 70 anos,4,11303.103793


# PERGUNTA 6:

### Escreva um comando em SQL que aplique um desconto de 10% em todas as vendas dos usuários que fizeram pelo menos 3 compras de produtos na mesma categoria, a partir da 4ª compra realizada. Exiba apenas os usuários que terão o desconto aplicado, mantendo todas as compras, o valor original e o valor com o desconto aplicado

In [120]:
consulta = '''SELECT cod_usuario, data_compra, num_compra, valor, valor_descontado FROM (
                SELECT *, CASE WHEN num_compra>=4 THEN CAST(ROUND(valor*0.9, 2) AS REAL) ELSE CAST(valor AS REAL) END AS valor_descontado FROM (
                  SELECT *, ROW_NUMBER() OVER(PARTITION BY cod_usuario) AS num_compra FROM (
                    SELECT * FROM (
                      SELECT * FROM vendas AS v
                      INNER JOIN (
                        SELECT DISTINCT cod_usuario FROM (
                          SELECT *, COUNT(categoria_produto) AS compras_por_categoria FROM (
                            SELECT cod_usuario, p.cod_produto, p.categoria_produto, data_compra, valor FROM (
                                SELECT * FROM vendas AS v
                                LEFT JOIN usuarios as u
                                ON v.cod_usuario=u.cod_usuario
                                ) AS t1
                            INNER JOIN produtos p
                            ON p.cod_produto=t1.cod_produto)
                          GROUP BY cod_usuario, categoria_produto)
                        WHERE compras_por_categoria >= 3) as t2
                      ON v.cod_usuario = t2.cod_usuario
                      ) AS t3
                    
                    INNER JOIN (
                      SELECT categoria_produto, cod_produto FROM produtos
                    ) AS t4
                    ON t4.cod_produto=t3.cod_produto
                    ORDER BY cod_usuario, data_compra)
                  )
                )
;
'''

cursor.execute(consulta)
pd.read_sql(consulta, conexao_sql)
                 

,cod_usuario,data_compra,num_compra,valor,valor_descontado
0,2,2021-04-14,1,219.57,219.57
1,2,2021-05-11,2,16198.20,16198.20
2,2,2021-05-17,3,8303.00,8303.00
3,2,2021-05-17,4,17997.30,16197.57
4,2,2021-06-08,5,1756.56,1580.90
...,...,...,...,...,...
6285,3403,2020-03-31,6,11898.30,10708.47
6286,3403,2020-06-30,7,47992.80,43193.52
6287,3403,2020-09-22,8,34.93,31.44
6288,3403,2021-01-15,9,1756.56,1580.90


In [42]:
consulta = '''SELECT DISTINCT nome_produto, cod_produto, categoria_produto FROM produtos;
'''

cursor.execute(consulta)
pd.read_sql(consulta, conexao_sql)

,nome_produto,cod_produto,categoria_produto
0,Notebook Asus Intel Core i7,1,Tecnologia
1,Bicicleta Caloi Aro 29,2,Lazer
2,Smartphone Samsung,3,Tecnologia
3,Mochila Targus,4,Material de escritorio
4,Escrivaninha em L,5,Material de escritorio
5,Sofa 3 lugares,6,Casa e bem-estar
6,Ar-condicionado 9500 BTUs LG,7,Casa e bem-estar
7,Cafeteira Nespresso,8,Eletrodomesticos
8,Microoondas 35L,9,Eletrodomesticos
9,Cadeira Gamer Xpress,10,Material de escritorio


# PERGUNTA 7

### Escreva um comando em SQL que retorna o produto, quantos usuários fizeram pelo menos uma compra desse produto e qual é o valor mínimo e máximo pago

In [173]:
consulta = '''
              SELECT nome_produto, num_usuarios, valor_minimo, valor_maximo FROM (
                SELECT *, COUNT(cod_usuario) as num_usuarios, MIN(valor) AS valor_minimo, MAX(valor) AS valor_maximo FROM (
                  SELECT * FROM vendas AS v
                  LEFT JOIN produtos AS p
                  ON v.cod_produto=p.cod_produto
                ) AS t1
                GROUP BY nome_produto
              );              
'''

cursor.execute(consulta)
pd.read_sql(consulta, conexao_sql)

,nome_produto,num_usuarios,valor_minimo,valor_maximo
0,Alcool em Gel 70% Johnson,976,4.99,99.8
1,Aparelho de Barbear OneBlade Philco,926,129.90,2598.0
2,Ar-condicionado 9500 BTUs LG,985,1300.49,26009.8
3,Bicicleta Caloi Aro 29,884,1345.78,26915.6
4,Biscoito Chocolicia Chocolate,934,5.00,100.0
5,Blusa Lacoste,925,138.99,2779.8
6,Cadeira Gamer Xpress,955,699.90,13998.0
7,Cafeteira Nespresso,953,219.57,4391.4
8,Escrivaninha em L,973,437.00,8740.0
9,Jogo Mortal Kombat 11 PS4,964,49.90,998.0


# PERGUNTA 8

### Escreva um comando em SQL que retorne o total de compras realizadas, o total de itens comprados e a receita total obtida

In [189]:
consulta = '''SELECT COUNT(data_compra) AS total_compras_realizadas, SUM(quantidade) AS total_itens_comprados, FORMAT(SUM(valor), 'N') AS receita_total_obtida 
              FROM vendas;              
'''

cursor.execute(consulta)
pd.read_sql(consulta, conexao_sql)

,total_compras_realizadas,total_itens_comprados,receita_total_obtida
0,20000,209149,215849001.66


# PERGUNTA 9

### Escreva um comando em SQL que retorne o produto, a quantidade de itens vendidos e a receita do produto que tem mais itens vendidos por ano e mês

In [211]:
consulta = '''SELECT nome_produto, ano_mes_compra, SUM(quantidade) AS itens_vendidos, SUM(valor) AS receita FROM (
                SELECT *, strftime('%Y-%m',data_compra) AS "ano_mes_compra" FROM vendas AS v
                INNER JOIN (
                    SELECT nome_produto, cod_produto, SUM(quantidade) AS itens_vendidos FROM (
                    SELECT * FROM vendas AS v
                    LEFT JOIN produtos AS p
                    ON v.cod_produto=p.cod_produto
                  )
                  GROUP BY nome_produto
                  ORDER BY itens_vendidos DESC LIMIT 1
                ) AS t1
                ON v.cod_produto=t1.cod_produto
              ) GROUP BY ano_mes_compra
              ORDER BY ano_mes_compra
;       
'''

cursor.execute(consulta)
pd.read_sql(consulta, conexao_sql)

,nome_produto,ano_mes_compra,itens_vendidos,receita
0,Microoondas 35L,2018-01,1,389.1
1,Microoondas 35L,2018-02,9,3501.9
2,Microoondas 35L,2018-03,31,12062.1
3,Microoondas 35L,2018-04,5,1945.5
4,Microoondas 35L,2018-06,28,10894.8
5,Microoondas 35L,2018-07,2,778.2
6,Microoondas 35L,2018-08,10,3891.0
7,Microoondas 35L,2018-09,49,19065.9
8,Microoondas 35L,2018-10,35,13618.5
9,Microoondas 35L,2018-11,20,7782.0




---
CONFERINDO

---





In [215]:
consulta = '''  SELECT nome_produto, cod_produto, SUM(quantidade) AS itens_vendidos, SUM(valor) FROM (
                SELECT * FROM vendas AS v
                LEFT JOIN produtos AS p
                ON v.cod_produto=p.cod_produto
              )
              GROUP BY nome_produto
              ORDER BY itens_vendidos DESC
;       
'''

cursor.execute(consulta)
pd.read_sql(consulta, conexao_sql)

,nome_produto,cod_produto,itens_vendidos,SUM(valor)
0,Microoondas 35L,9,10478,4076989.80
1,Mochila Targus,4,10384,611617.60
2,SPA Intel 700 L,21,10320,61910712.00
3,SmartTV Samsung 4K 55 polegadas,19,10280,25698972.00
4,Ar-condicionado 9500 BTUs LG,7,10279,13367736.71
5,Lenço umedecido Turma da Monica,18,10277,81188.30
6,Escrivaninha em L,5,10237,4473569.00
7,Monitor LG 19 pol curvado,17,10171,5940881.10
8,Alcool em Gel 70% Johnson,15,10150,50648.50
9,Sofa 3 lugares,6,10145,9129485.50


In [218]:
consulta = '''
              SELECT SUM(receita) FROM (
              SELECT nome_produto, ano_mes_compra, SUM(quantidade) AS itens_vendidos, SUM(valor) AS receita FROM (
                SELECT *, strftime('%Y-%m',data_compra) AS "ano_mes_compra" FROM vendas AS v
                INNER JOIN (
                    SELECT nome_produto, cod_produto, SUM(quantidade) AS itens_vendidos FROM (
                    SELECT * FROM vendas AS v
                    LEFT JOIN produtos AS p
                    ON v.cod_produto=p.cod_produto
                  )
                  GROUP BY nome_produto
                  ORDER BY itens_vendidos DESC LIMIT 1
                ) AS t1
                ON v.cod_produto=t1.cod_produto
              ) GROUP BY ano_mes_compra
              ORDER BY ano_mes_compra
              )

;       
'''
cursor.execute(consulta)
pd.read_sql(consulta, conexao_sql)

,SUM(receita)
0,4076989.8


# PERGUNTA 10

### Escreva um comando em SQL que exiba quem são os usuários que compraram produtos diferentes

# PERGUNTA 11

### Escreva um comando em SQL que verifique usuários que não fizeram nenhuma compra